In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import pandas as pd
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
from torch.nn import init
import torch
import torchvision
import torch.nn as nn



import torchvision.models as models
from torchvision import transforms as T

2024-06-24 11:44:18.585367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 11:44:18.585487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 11:44:18.718250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import timm

In [3]:
path = "/kaggle/input/autism-paper-data/Retained/dataset_file_directory.csv"
df_original = pd.read_csv(path)
df_original.head()


,Filename,Participant,Label
0,210408_2025_00-01-38.27--00-01-40.84.wav,P01,social
1,210324_2036_00-06-03.61--00-06-05.78.wav,P01,social
2,210324_2036_00-09-04.66--00-09-06.16.wav,P01,social
3,210324_2036_00-11-18.6--00-11-20.3.wav,P01,social
4,200506_2110_00-01-25.92--00-01-26.58c.wav,P01,social


In [4]:
df_original.shape[0]

6450

In [5]:
df_original.Label.unique() #Print unique types of sound

array(['social', 'selftalk', 'request', 'delighted', 'dysregulated',
       'frustrated'], dtype=object)

In [6]:
df_original['Label'].value_counts() #Number of datapoints available for each target variable

Label
selftalk        1885
frustrated      1536
delighted       1272
dysregulated     704
social           634
request          419
Name: count, dtype: int64

In [7]:
df_original["classID"] = df_original['Label']
df_original['classID'].replace(['selftalk', 'frustrated', 'delighted', 'dysregulated', 'social', 'request'],
                        [0, 1, 2, 3, 4, 5], inplace=True)


/tmp/ipykernel_34/1495406677.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_original['classID'].replace(['selftalk', 'frustrated', 'delighted', 'dysregulated', 'social', 'request'],
/tmp/ipykernel_34/1495406677.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_original['classID'].replac

In [8]:
df = df_original
df = df.drop(['Participant', 'Label'], axis=1)
df.head()


,Filename,classID
0,210408_2025_00-01-38.27--00-01-40.84.wav,4
1,210324_2036_00-06-03.61--00-06-05.78.wav,4
2,210324_2036_00-09-04.66--00-09-06.16.wav,4
3,210324_2036_00-11-18.6--00-11-20.3.wav,4
4,200506_2110_00-01-25.92--00-01-26.58c.wav,4


In [9]:
df['classID'].value_counts() #Number of datapoints available for each target variable

classID
0    1885
1    1536
2    1272
3     704
4     634
5     419
Name: count, dtype: int64

In [10]:
# Construct file path by concatenating fold and file name
df['relative_path'] = '/' + df['Filename'].astype(str)

# Take relevant columns
df = df[['relative_path', 'classID']]
df.head()

,relative_path,classID
0,/210408_2025_00-01-38.27--00-01-40.84.wav,4
1,/210324_2036_00-06-03.61--00-06-05.78.wav,4
2,/210324_2036_00-09-04.66--00-09-06.16.wav,4
3,/210324_2036_00-11-18.6--00-11-20.3.wav,4
4,/200506_2110_00-01-25.92--00-01-26.58c.wav,4


In [11]:
#Random shuffle of rows.
df = df. sample(frac=1)
df.head()

,relative_path,classID
2945,/200929_0101_00-00-26.37--00-00-27.40c.wav,0
7,/200506_2110_00-02-55.52--00-02-58.79.wav,4
6125,/210113_1909_00-03-55.66--00-03-56.11.wav,1
1196,/200229_2156_00-03-55.9--00-03-56.29.wav,2
4422,/200906_1111_00-00-25.06--00-00-25.774c.wav,4


In [12]:

# Audio utility functions
class AudioUtil():
    @staticmethod
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)

    @staticmethod
    def rechannel(aud, new_channel):
        sig, sr = aud
        if sig.shape[0] == new_channel:
            return aud
        if new_channel == 1:
            resig = sig[:1, :]
        else:
            resig = torch.cat([sig, sig])
        return (resig, sr)

    @staticmethod
    def resample(aud, newsr):
        sig, sr = aud
        if sr == newsr:
            return aud
        num_channels = sig.shape[0]
        resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1, :])
        if num_channels > 1:
            retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:, :])
            resig = torch.cat([resig, retwo])
        return (resig, newsr)

    @staticmethod
    def pad_trunc(aud, max_ms):
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr // 1000 * max_ms
        if sig_len > max_len:
            sig = sig[:, :max_len]
        elif sig_len < max_len:
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))
            sig = torch.cat((pad_begin, sig, pad_end), 1)
        return (sig, sr)

    @staticmethod
    def time_shift(aud, shift_limit):
        sig, sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)

    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig, sr = aud
        top_db = 80
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return (spec)

    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec
        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)
        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)
        return aug_spec


In [13]:
import torchvision


In [14]:
# Sound dataset class
class SoundDS(Dataset):
    def __init__(self, df, data_path):
        self.df = df
        self.data_path = str(data_path)
        self.duration = 4000
        self.sr = 44100
        self.channel = 2
        self.shift_pct = 0.4

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_file = self.data_path + self.df.loc[idx, 'relative_path']
        class_id = self.df.loc[idx, 'classID']
        aud = AudioUtil.open(audio_file)
        reaud = AudioUtil.resample(aud, self.sr)
        rechan = AudioUtil.rechannel(reaud, self.channel)
        dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
        shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
        sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
        aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
        aug_sgram = torchvision.transforms.Resize((224, 224))(aug_sgram)
        return aug_sgram, class_id


In [15]:
data_path = "/kaggle/input/autism-paper-data/Retained/audio"
myds = SoundDS(df, data_path)
#myds = SoundDS(df, path)

#Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

#Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

In [16]:
print(num_items, num_train, num_val)

6450 5160 1290


In [17]:
type(train_dl)

torch.utils.data.dataloader.DataLoader

In [18]:
from torchvision import models

In [19]:
# Load and modify MobileNetV2 model
model = torchvision.models.mobilenet_v2(pretrained=True)
model.features[0][0] = nn.Conv2d(2, model.features[0][0].out_channels,
                                  kernel_size=model.features[0][0].kernel_size,
                                  stride=model.features[0][0].stride,
                                  padding=model.features[0][0].padding)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, df_original['classID'].nunique())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = model.to(device)

# Check that it is on Cuda
next(myModel.parameters()).device

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 149MB/s]


device(type='cuda', index=0)

In [20]:
def training(model, train_dl, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                    steps_per_epoch=int(len(train_dl)),
                                                    epochs=num_epochs,
                                                    anneal_strategy='linear')

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        for i, data in enumerate(train_dl):
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()
            _, prediction = torch.max(outputs, 1)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction / total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

    print('Finished Training')

In [21]:
def inference(model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    with torch.no_grad():
        for data in val_dl:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            outputs = model(inputs)
            _, prediction = torch.max(outputs, 1)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction / total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')


In [22]:
print(next(myModel.parameters()).device)

cuda:0


In [23]:
num_epochs = 50
training(myModel, train_dl, num_epochs)
inference(myModel, val_dl)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 0, Loss: 1.58, Accuracy: 0.35
Epoch: 1, Loss: 1.41, Accuracy: 0.44
Epoch: 2, Loss: 1.33, Accuracy: 0.48
Epoch: 3, Loss: 1.26, Accuracy: 0.53
Epoch: 4, Loss: 1.21, Accuracy: 0.55
Epoch: 5, Loss: 1.18, Accuracy: 0.57
Epoch: 6, Loss: 1.16, Accuracy: 0.57
Epoch: 7, Loss: 1.12, Accuracy: 0.59
Epoch: 8, Loss: 1.08, Accuracy: 0.60
Epoch: 9, Loss: 1.06, Accuracy: 0.61
Epoch: 10, Loss: 1.05, Accuracy: 0.61
Epoch: 11, Loss: 1.03, Accuracy: 0.63
Epoch: 12, Loss: 1.03, Accuracy: 0.64
Epoch: 13, Loss: 1.02, Accuracy: 0.62
Epoch: 14, Loss: 1.02, Accuracy: 0.64
Epoch: 15, Loss: 1.00, Accuracy: 0.63
Epoch: 16, Loss: 0.97, Accuracy: 0.65
Epoch: 17, Loss: 0.94, Accuracy: 0.65
Epoch: 18, Loss: 0.90, Accuracy: 0.67
Epoch: 19, Loss: 0.88, Accuracy: 0.68
Epoch: 20, Loss: 0.87, Accuracy: 0.69
Epoch: 21, Loss: 0.82, Accuracy: 0.71
Epoch: 22, Loss: 0.80, Accuracy: 0.71
Epoch: 23, Loss: 0.78, Accuracy: 0.71
Epoch: 24, Loss: 0.74, Accuracy: 0.73
Epoch: 25, Loss: 0.73, Accuracy: 0.74
Epoch: 26, Loss: 0.69,

In [24]:
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

def inference(model, val_dl):
    all_predictions = []
    all_labels = []
    
    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    f1 = f1_score(all_labels, all_predictions, average='weighted')
        
    print(f'Accuracy: {accuracy:.2f}, F1 Score: {f1:.2f}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Accuracy: 0.71, F1 Score: 0.71
